In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error
import random
import math
import datetime


## Load the aggregated data

In [2]:
#output of notebook 5a
%time df=pd.read_csv("./tmplocaldata/final/groupbydestn/pickup_dropoff_aggregated_with_feat_ext.csv")

Wall time: 25.3 s


In [3]:
df.head()

,pickup_geohash,dropoff_geohash,time_num,day_of_week,count,time_sin,time_cos,pickup_lat,pickup_long,dropoff_lat,dropoff_long
0,dr5rtk,dr5rsj,0.416667,4,2,0.500000,-0.866025,40.718079,-73.943481,40.723572,-73.998413
1,dr5x0z,dr5xcf,0.683333,0,2,-0.913545,-0.406737,40.646667,-73.789673,40.751038,-73.745728
2,dr72rd,dr782h,0.650000,3,2,-0.809017,-0.587785,40.838928,-73.844604,40.849915,-73.822632
3,dr5rvn,dr5rt5,0.083333,2,11,0.500000,0.866025,40.773010,-73.954468,40.712585,-73.954468
4,dr72jd,dr72m3,0.050000,0,3,0.309017,0.951057,40.794983,-73.932495,40.833435,-73.943481


## Prepare data for Machinel Learning

##### Convert the target variable to log scale

In [4]:
df['log_count'] = np.log10(df['count']+1)

In [5]:
trainSet, testSet = train_test_split(df, train_size=0.8, random_state=56)
del df

In [6]:
X_train = trainSet[['time_num', 'time_sin', 'time_cos','day_of_week', 'pickup_lat', 'pickup_long', 'dropoff_lat', 'dropoff_long']]
#y_train = trainSet['count']
y_train = trainSet['log_count']

X_test = testSet[['time_num', 'time_sin', 'time_cos','day_of_week', 'pickup_lat', 'pickup_long', 'dropoff_lat', 'dropoff_long']]
#y_test = testSet['count']
y_test = testSet['log_count']

del trainSet
del testSet

## Machine learning

In [7]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print "Best score: ", gs.best_score_
    best = gs.best_estimator_
    return best

In [8]:
estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)

In [9]:
%%time
# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [20],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [20]}
best = cv_optimize(estimator, parameters, X_train, y_train, n_folds=2, verbose=3)
#best = cv_optimize(estimator, parameters, X_train, y_train, n_folds=2, score_func='mean_squared_error', verbose=3)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] max_features=auto, n_estimators=20, max_depth=20 ................
[CV]  max_features=auto, n_estimators=20, max_depth=20, score=0.869117 - 3.7min
[CV] max_features=auto, n_estimators=20, max_depth=20 ................
[CV]  max_features=auto, n_estimators=20, max_depth=20, score=0.868532 - 3.9min

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  3.7min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.6min finished



BEST {'max_features': 'auto', 'n_estimators': 20, 'max_depth': 20} 0.868824283087 [mean: 0.86882, std: 0.00029, params: {'max_features': 'auto', 'n_estimators': 20, 'max_depth': 20}] <function _passthrough_scorer at 0x000000002052D7B8>
Best score:  0.868824283087
Wall time: 15min 27s


## Evaluate the results

In [11]:
%%time
rmse = np.sqrt(mean_squared_error(best.predict(X_test),y_test))
print "RMSE = %0.3f (this is in log-space!)" % rmse
print "RMSE in real scale: %0.2f" % np.power(10,rmse)

RMSE = 0.204 (this is in log-space!)
RMSE in real scale: 1.60
Wall time: 6.09 s


In [12]:
%%time

# Fit the best Random Forest and calculate R^2 values for training and test sets

#reg=best.fit(X_train, y_train)
training_accuracy = best.score(X_train, y_train)
test_accuracy = best.score(X_test, y_test)
print "############# based on standard predict ################"
print "R^2 on training data: %0.4f" % (training_accuracy)
print "R^2 on test data:     %0.4f" % (test_accuracy)

############# based on standard predict ################
R^2 on training data: 0.8767
R^2 on test data:     0.8674
Wall time: 30.4 s


In [13]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
np.round(np.power(10,np.column_stack((best.predict(X_test),y_test))) - 1,decimals=0).astype(int)

array([[  7,  12],
       [  1,   1],
       [181, 708],
       ..., 
       [  7,   7],
       [130, 327],
       [151, 714]])

## Create output for Viz

In [45]:
X_test["predicted_pickups"] = np.round(np.power(10,best.predict(X_test))-1, decimals=0)
X_test["actual_pickups"] =  np.round(np.power(10,y_test)-1, decimals=0)

In [47]:
outputDF = X_test[["time_num", "day_of_week", "pickup_lat", "pickup_long","dropoff_lat","dropoff_long","predicted_pickups","actual_pickups"]]
outputDF.head()

,time_num,day_of_week,pickup_lat,pickup_long,dropoff_lat,dropoff_long,predicted_pickups,actual_pickups
8981259,0.883333,3,40.751038,-73.998413,40.844421,-73.932495,7,12
1559136,0.816667,3,40.641174,-73.778687,40.635681,-73.778687,1,1
4881940,0.183333,5,40.745544,-73.888550,40.751038,-73.866577,181,708
160601,0.816667,1,40.745544,-73.943481,40.690613,-73.844604,1,1
659255,0.216667,4,40.789490,-73.943481,40.756531,-73.998413,11,14


In [48]:
outputDF.to_csv('./tmplocaldata/final/groupbydestn_only_locn/pickup_dropoffs.csv', index=False)

## Appendix

In [14]:
import operator
dict_feat_imp = dict(zip(list(X_train.columns.values),best.feature_importances_))

sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('dropoff_long', 0.30408825024272568),
 ('pickup_long', 0.23172967883312015),
 ('dropoff_lat', 0.22555554859312843),
 ('pickup_lat', 0.21258320917845941),
 ('time_cos', 0.019245191836231027),
 ('time_num', 0.0046247418564218839),
 ('time_sin', 0.0014808130062024325),
 ('day_of_week', 0.00069256645371093648)]

In [11]:
# numbers in normal scale
%%time
print "RMSE= ",np.sqrt(mean_squared_error(best.predict(X_test),y_test))

RMSE=  66.189331534
Wall time: 6.21 s


In [8]:
# Latitude  range from -90 to +90
# Longitide  range from -180 to +180
def remap(oldValue, oldMin, oldMax, newMin, newMax):
    oldRange = (oldMax - oldMin)
    newRange = (newMax - newMin)
    newValue = (((oldValue - oldMin) * newRange) / oldRange) + newMin
    return newValue



## Test remapping values from one range to another

In [14]:
#test
# lat:40.718079, long: -73.943481
originalLong = -73.943481
rescaledLong = remap(-73.943481, -180, 180, -90,90)
print rescaledLong
scaledbackLong = remap(rescaledLong, -90,90, -180, 180)
print scaledbackLong
originalLong- scaledbackLong

-36.9717405
-73.943481


0.0

In [18]:
originalLong = -73.943481
rescaledLong = np.interp(originalLong, [-180,180], [-90,90])
scaledbackLong = np.interp(rescaledLong, [-90,90], [-180,180])
print scaledbackLong
originalLong- scaledbackLong

-73.943481


0.0